# Example 1

In [ ]:
%%file busy_loop.pyx
cpdef busy_loop():
    cdef:
        long i, total = 0, n = 10000
    for i in range(n):
        total += i
    return total

In [ ]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup (
    name = "busy_loop",
    ext_modules = cythonize("busy_loop.pyx")
)

In [ ]:
%run setup.py build_ext --inplace

In [ ]:
def py_busy_loop():
    n = 100000
    total = 0
    for i in range(n):
        total += i
    return total

In [ ]:
import busy_loop
%timeit -n 1000 busy_loop.busy_loop()
%timeit -n 1000 py_busy_loop()

# Example 2

In [ ]:
%%file fibonacci.pyx
cpdef fibonacci(long long n):
    cdef:
        long long current = 1, previous = 1, temp = 0, i
    if n > 2:
        for i in range(2, n):
            temp = current
            current += previous
            previous = temp
    else:
        return current
    return current

In [ ]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup (
    name = "fibonacci",
    ext_modules = cythonize("fibonacci.pyx")
)

In [ ]:
%run setup.py build_ext --inplace

In [ ]:
def py_fibonacci(n):
    current = 1
    previous = 1
    temp = 0
    if n > 2:
        for i in range(2, n):
            temp = current
            current += previous
            previous = temp
    else:
        return current
    return current

In [ ]:
import fibonacci as fib
%timeit -n 10000 fib.fibonacci(92)
%timeit -n 10000 py_fibonacci(92)

In [ ]:
print(fib.fibonacci(92))
print("_________________________________________________")
print(py_fibonacci(92))

In [ ]:
print(fib.fibonacci(93))
print("_________________________________________________")
print(py_fibonacci(93))

# cProfile example

In [ ]:
%%file py_test.py
import math as mt
def integrate(a, b, f, N = 10000000):
    dx = (b - a) / N
    s = 0.0
    for i in range(N):
        s += f(a+i*dx)
    return s * dx

def some_func(a):
    return mt.sin(a) * mt.sqrt((mt.exp(-a)))
        
def main():
    total = 0
    a = 2
    b = 7
    
    #Find the integral of some_func from a to b
    result = integrate(a, b, some_func)
    return result
    
    
if __name__ == '__main__':
    import cProfile
    cProfile.run('main()', sort = 'time')

In [ ]:
%run py_test.py

In [ ]:
%%file py_test_ext.pyx
#cython: profile = True
from libc.math cimport sin, sqrt, exp
cpdef integrate(double a, double b, f,long long N = 10000000):
    cdef:
        double dx = (b - a) / N
        double s = 0.0
        long long i
    for i in range(N):
        s += f(a+i*dx)
    return s * dx

cpdef some_func(double a):
    return sin(a) * sqrt((exp(-a)))

In [ ]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(
    name = "py_test_ext",
    ext_modules = cythonize("py_test_ext.pyx")
)

In [ ]:
%run setup.py build_ext --inplace

In [ ]:
%%file py_test2.py
from py_test_ext import integrate, some_func
import math as mt

def main():
    total = 0
    a = 2
    b = 7
    
    #Find the integral of some_func from a to b
    result = integrate(a, b, some_func)
    return result
    
    
if __name__ == '__main__':
    import cProfile
    cProfile.run('main()', sort = 'time')

In [ ]:
%run py_test2.py